In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import display
pd.set_option('display.max_columns', None)

In [2]:
#loading dataset
df = pd.read_csv('data\AMZN.csv')

#preprocessing
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# General Functions

## API Call

In [37]:
def AV_daily_call(symbol, output_size='compact'):
    # symbol = ticker
    # output_size = 'compact'(last 100 days) or 'full'(full-length)
    
    function = 'TIME_SERIES_DAILY'
    api_key = '1W0X0E2POGJ6T74Z'

    url = 'https://www.alphavantage.co/query?function={}&symbol={}&outputsize={}&apikey={}'.format(
                    function, symbol, output_size, api_key)

    # grabs correct info into dataframe
    response = requests.get(url)
    response_dict = json.loads(response.text)
    data = response_dict['Time Series (Daily)']
    df = pd.DataFrame.from_dict(data)
    
    # Fixes dataframe into correct format
    df = df.T
    df = df.iloc[::-1]
    
    df = df.astype(float)
    df.index = pd.to_datetime(df.index)
    df.rename_axis(symbol, inplace=True)
    
    df.columns = (['Open', 'High', 'Low', 'Close', 'Volume'])
    
    return df

## add row

In [3]:
def add_row(df, Open, High, Low, Close):
    index = len(df.index)
    values = [Open, High, Low, Close]
    
    missing_values = len(df.columns) - 4
    
    for i in range(missing_values):
        values.append(np.nan)
    
    df.loc[index] = values  

# Indicators

## Simple Moving Average (SMA)

In [8]:
def sma(df, i, n=9):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    # n = variable determines length of algorithm 
    
    if 'SMA' not in df.columns:
        df['SMA'] = np.nan
        df['SMA_diff'] = np.nan
        
    if i in range(0,(n-1)):
        df.iloc[i, df.columns.get_loc('SMA')] = np.nan
    
    sma = df.iloc[(i-(n-1)):i+1]['Close'].mean()
    df.iloc[i, df.columns.get_loc('SMA')] = sma
    
    diff = sma - df.iloc[i-1]['SMA']
    df.iloc[i, df.columns.get_loc('SMA_diff')] = diff

## Stochastic Oscillator

Stochastic = (C-L_n)/(H_n-L_n)*100   
>C = The Most recent closing price  
>L_n = The lowest price traded of the n previous trading sessions  
>H_n = The highest price traded during the same n period  

In [2]:
def stochastic(df, i, n=14):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    # n = variable determines length of algorithm 
    
    
    if 'Stochastic' not in df.columns:
        df['Stochastic'] = np.nan
        df['Stochastic_diff'] = np.nan
        
    if i in range(0, n-1):
        df.iloc[i, df.columns.get_loc('Stochastic')] = np.nan
        return 
        
    C = df.iloc[i]['Close']
    Lowest = min(list(df.iloc[i-(n-1): i+1]['Low']))
    Highest = max(list(df.iloc[i-(n-1): i+1]['High']))
        
    value = ((C - Lowest) / (Highest - Lowest)) * 100
        
    df.iloc[i, df.columns.get_loc('Stochastic')] = value 
    
    diff = value - df.iloc[i-1]['Stochastic']
    df.iloc[i, df.columns.get_loc('Stochastic_diff')] = diff 

## Relative Strength Index(RSI)

>RSI = 100 - (100/RS + 1)  
>RS = Average Gain  / Average Loss  
>Average Gain = Sum of Gains over the past 14 periods / 14  
>Average Loss = Sum of Losses over the past 14 periods / 14  

>>Note: Tradingview.com uses ema to calculate Average Gain and Loss, Values will differ

### rsi variables

In [3]:
def rsi_vars(df,i):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    
    if 'rsi_up_var' not in df.columns:
        df['rsi_up_var'] = np.nan
        
    if 'rsi_down_var' not in df.columns:
        df['rsi_down_var'] = np.nan
        
    if i == 0:
        return
    
    current_close = df.iloc[i]['Close']
    previous_close = df.iloc[i-1]['Close']
        
    if current_close > previous_close:
        df.iloc[i, df.columns.get_loc('rsi_up_var')] = current_close - previous_close
        df.iloc[i, df.columns.get_loc('rsi_down_var')] = 0
        
    else:
        df.iloc[i, df.columns.get_loc('rsi_up_var')] = 0
        df.iloc[i, df.columns.get_loc('rsi_down_var')] = previous_close - current_close

### rsi function

In [4]:
def rsi(df, i, n=14):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    # n = variable determines length of algorithm 
    
    rsi_vars(df,i)
    
    if 'RSI' not in df.columns:
        df['RSI'] = np.nan
        df['RSI_diff'] = np.nan
        
    if i > n-1:
        avg_up_move = df['rsi_up_var'].iloc[i-(n-1):i+1].mean()
        avg_down_move = df['rsi_down_var'].iloc[i-(n-1):i+1].mean()
            
        relative_strength = avg_up_move / avg_down_move
        rsi = 100-(100/(1 + relative_strength))
            
        df.iloc[i, df.columns.get_loc('RSI')] = rsi
        
        diff = rsi - df.iloc[i-1]['RSI']
        df.iloc[i, df.columns.get_loc('RSI_diff')] = diff 
    

## Rate Of Change (ROC)

ROC = ((Closing Price p - Closing Price p-n) / Closing Price p-n) * 100

>Closing Price p = Closing price of most recent period  
>Closing Price p-n = Closing price n periods before the most recent period

In [5]:
def roc(df, i, n=9):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    # n = variable determines length of algorithm 
    
    if 'ROC' not in df.columns:
        df['ROC'] = np.nan
        df['ROC_diff'] = np.nan
        
    if i in range(0, n):
        return
    
    current_close = df.iloc[i]['Close']
    n_close = df.iloc[i - n]['Close']
        
    roc = ((current_close - n_close)/n_close) * 100
    
    df.iloc[i, df.columns.get_loc('ROC')] = roc
    
    diff = roc - df.iloc[i-1]['ROC']
    df.iloc[i, df.columns.get_loc('ROC_diff')] = diff 

## Average True Range (ATR)

ATR = average TR from current to past n bars  
> TR = Max((High-Low),   ABS(High - Previous Close),   ABS(Low - Previous Close))

### true range

In [9]:
def true_range(df,i):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    
    if 'tr' not in df.columns:
        df['tr'] = np.nan
    
    if i == 0:
        return
    
    current_high = df.iloc[i]['High']
    current_low = df.iloc[i]['Low']
    pre_close = df.iloc[i-1]['Close']
        
    method_1 = abs(current_high - current_low)
    method_2 = abs(current_high - pre_close)
    method_3 = abs(current_low - pre_close)
        
    TR = max(method_1, method_2, method_3)
    
    df.iloc[i, df.columns.get_loc('tr')] = TR

### atr function

In [6]:
def atr(df,i,n=14):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    # n = variable determines length of algorithm 
    
    true_range(df,i)
    
    if 'ATR' not in df.columns:
        df['ATR'] = np.nan
        df['ATR_diff'] = np.nan
    
    if i > n-1:
        ATR = df.iloc[i-(n-1) : i+1]['tr'].mean()
        
        df.iloc[i, df.columns.get_loc('ATR')] = ATR
        
        diff = ATR - df.iloc[i-1]['ATR']
        df.iloc[i, df.columns.get_loc('ATR_diff')] = diff 

## Average Directional Index (ADX)

ADX = avg((i) DX bars)  
DX = (abs(positive DMI - negative DMI) / (positive DMI + negative DMI)) * 100

> i = bar count  

> positive DMI = avg(positive DX)  
> negative DMI = avg(negative DX)  

> positive DX = ((if high diff > low diff) and (if high diff is > 0)) = high diff, (else) = 0  
> negative DX = ((if low diff > high diff) and (if low diff is > 0)) = low diff, (else) = 0  

> high diff = current high - previous high  
> low diff = current low - previous low

>>Note: Trading view uses EMA over simple moving averages, values may differ.

### adx variables

In [11]:
def adx_vars(df,i):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index

    if 'pos_dx' not in df.columns:
        df['pos_dx'] = np.nan
    
    if 'neg_dx' not in df.columns:
        df['neg_dx'] = np.nan
        
    if i == 0:
        return
    
    #differencing the highs and lows
    current_high = df.iloc[i]['High']
    previous_high = df.iloc[i-1]['High']
    
    current_low = df.iloc[i]['Low']
    previous_low = df.iloc[i-1]['Low']
    
    high_diff = current_high - previous_high
    low_diff = current_low - previous_low
    
    #calculate positive and negative DX
    if (high_diff > low_diff) and (high_diff > 0):
        
        df.iloc[i, df.columns.get_loc('pos_dx')] = high_diff
        df.iloc[i, df.columns.get_loc('neg_dx')] = 0
        
    elif(low_diff > high_diff) and (low_diff > 0):
      
        df.iloc[i, df.columns.get_loc('pos_dx')] = 0
        df.iloc[i, df.columns.get_loc('neg_dx')] = low_diff
        
    else:
        df.iloc[i, df.columns.get_loc('pos_dx')] = 0
        df.iloc[i, df.columns.get_loc('neg_dx')] = 0

### adx function

In [7]:
def adx(df,i,n=14):
    # df = dataframe, columns must include ['Open', 'High', 'Low', 'Close']
    # i = dataframe index
    # n = variable determines length of algorithm
    
    adx_vars(df,i)
    
    if 'ATR' not in df.columns:
        atr(df,i,n)
        
    if 'dx' not in df.columns:
        df['dx'] = np.nan
    
    if 'ADX' not in df.columns:
        df['ADX'] = np.nan
        df['ADX_diff'] = np.nan
    
    if i > n-1:
        # calculating smooth pos, neg dx
        avg_pos_dx = df.iloc[i-(n-1):i+1]['pos_dx'].mean()
        avg_neg_dx = df.iloc[i-(n-1):i+1]['neg_dx'].mean()
        
        #calculating pos, neg DMI
        ATR = df.iloc[i]['ATR']
        pos_dmi = (avg_pos_dx / ATR) * 100
        neg_dmi = (avg_neg_dx / ATR) * 100
        
        #calculating direction index (dx)
        dx = (abs(pos_dmi - neg_dmi)/ (pos_dmi + neg_dmi))*100
        df.iloc[i, df.columns.get_loc('dx')] = dx
        
        if i > n*2-2:
            adx = df.iloc[i-13:i+1]['dx'].mean()
            df.iloc[i, df.columns.get_loc('ADX')] = adx 
            
            diff = adx - df.iloc[i-1]['ADX']
            df.iloc[i, df.columns.get_loc('ADX_diff')] = diff 

# Test

In [30]:
for i in range(len(df)):
    sma(df, i)
    stochastic(df, i)
    rsi(df,i)
    roc(df,i)
    atr(df,i)
    adx(df,i)

In [31]:
df.tail(10)

,Open,High,Low,Close,SMA,Stochastic,rsi_up_var,rsi_down_var,RSI,ROC,tr,ATR,pos_dx,neg_dx,dx,ADX
Date,,,,,,,,,,,,,,,,
2021-01-20,3181.989990,3279.800049,3175.000000,3263.379883,3151.294407,69.261935,142.619873,0.000000,45.367025,3.982947,159.040039,71.951433,134.800049,0.000000,60.093352,20.663963
2021-01-21,3293.000000,3348.550049,3289.570068,3306.989990,3167.386637,84.170615,43.610107,0.000000,51.651295,4.580100,85.170166,73.775722,0.000000,114.570068,7.433701,20.475003
2021-01-22,3304.310059,3321.909912,3283.159912,3292.229980,3179.556641,78.548826,0.000000,14.760010,52.819762,3.441419,38.750000,73.354283,0.000000,0.000000,7.433701,20.286043
2021-01-25,3328.500000,3363.889893,3243.149902,3294.000000,3199.533312,74.849790,1.770020,0.000000,59.631159,5.773215,120.739991,72.837141,41.979981,0.000000,17.488112,20.815255
2021-01-26,3296.360107,3338.000000,3282.870117,3326.129883,3222.344401,86.411881,32.129883,0.000000,59.649239,6.578372,55.129883,72.609288,0.000000,39.720215,12.071382,20.709853
2021-01-27,3341.489990,3346.520020,3207.080078,3232.580078,3229.754422,52.747538,0.000000,93.549805,58.247554,2.106523,139.439942,76.329991,8.520020,0.000000,13.882286,20.809390
2021-01-28,3235.040039,3301.679932,3228.689941,3237.620117,3241.993327,54.561220,5.040039,0.000000,56.830961,3.522021,72.989991,76.532122,0.000000,21.609863,14.499558,19.089216
2021-01-29,3230.000000,3236.989990,3184.550049,3206.199951,3253.321099,43.254524,0.000000,31.420166,52.086220,3.284206,53.070068,76.862845,0.000000,0.000000,14.499558,18.284019
2021-02-01,3242.360107,3350.260010,3235.030029,3342.879883,3278.001085,92.439448,136.679932,0.000000,68.107889,7.117493,144.060059,81.959996,113.270020,0.000000,32.955970,18.797137


### add_row Test

In [15]:
o = 1111
h = 2222
l = 3333
c = 4444

# add_row(df,o,h,l,c)

# i = len(df)-1

# sma(df, i)

# df

In [16]:
min([1,2,34])

1